<a href="https://colab.research.google.com/github/takeshitayy/openai-whisper/blob/main/openai_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【OpenAI】Whisper で音声データを文章化し、発話の開始時間と終了時間を含めたCSVデータにする
- OpenAIが公開している [Wisper](https://github.com/openai/whisper/) を利用して、音声データを文章化する
- 文章化するとある程度で区切られて出力され、その区切られた文章の開始時間と終了時間を含めCSVデータにする

## 実行方法

1. ランタイムの設定
    1. メニューバーの [ランタイム] - [ランタイムのタイプの変更] をクリック
    ![ランタイムのタイプの変更](https://docs.google.com/drawings/d/e/2PACX-1vTo2XkKxxwux9ATDbSPQI0PbJHMYzTJPKwUcUtIj8UwlT_h4sFm9vJtdjusib3p0nNX-j7lYX0EyWu1/pub?w=601&h=545)

    1. ハードウェア アクセラレータ で `GPU` を設定する
    ![ランタイムのタイプの変更](https://docs.google.com/drawings/d/e/2PACX-1vQL7egTWuLu9NOLb5-Z1uZCwIa9dMKraLhn6T6B3KO0_GdYhawWdOyKQaE0SlBDZ0gDr1xAUp0aCPAQ/pub?w=644&h=340)

1. 音声データの追加
    1. サイドバーの 📁アイコン をクリックしてファイルの一覧を表示し、テキストを抽出したい 音声データ を drag & drop で追加
    ![音声データの追加](https://docs.google.com/drawings/d/e/2PACX-1vQU_evi3lR57aAgoRfJhAr5FYpGBf10uA85v8Dx25rE-HMAauXwX1ON0o9KcBVyXvUjqR4H8_rMadCM/pub?w=525&h=307)

1. 変数のセット
    1. 追加した 音声データ を右クリックして `パスをコピー` でファイルの絶対パスをコピーする
        ![音声データのフルパスのコピー](https://docs.google.com/drawings/d/e/2PACX-1vR6sCrnZT8S9QNnm7IRGZRhGl1cP3AOd8ucT0iIxx-azlz_S3IoLkv1sAALxKGkc1z7JWXLaz9U2bcY/pub?w=447&h=384)
    1. 変数 `audioPath` に音声データの絶対パスをセット
    1. 変数 `outputCsvName` に出力するcsvのファイル名をセット
    1. 変数 `model`で 使用する whisper のモデルを選択する

![変数のセット](https://docs.google.com/drawings/d/e/2PACX-1vSw82EkS6M7f6H1Kp3MRyeEbf2PqNHzbWnVitCgKwAnD10z2dLsG3NL-ENrHnHzHG1PpWKUZSwxJ5iE/pub?w=946&h=393)

1. `Ctrl + F9` ですべてのセルを実行すると `/content/` 以下に指定した名前のcsvファイルが出力される

### 変数のセットとパッケージの導入

In [6]:
#@title 変数のセット
#@markdown 音声データの絶対パス:
audioPath = "/content/audio.mp3" #@param {type:"string"}

#@markdown 出力するCSVのファイル名（/content/ 以下に出力される）:
outputCsvName = "output.csv" #@param {type:"string"}

#@markdown whisper のモデル:
model = "large" #@param ["tiny", "base", "small", "medium", "large"]

import os
outputCsvPath = os.path.join('/content/', outputCsvName)

#### パッケージの導入

In [7]:
! pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-arrtgit1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-arrtgit1
  Resolved https://github.com/openai/whisper.git to commit b5851c6c40e753606765ac45b85b298e3ae9e00d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 音声データを文章化
Whisper で音声データの文章化を行うと以下の dict が返ってくる  

**language**: 検出された言語  
**text**: 文章全体  
**segments**: 分割された文章の詳細情報（その文章の開始時間や終了時間の情報も含む）


In [8]:
import whisper
model = whisper.load_model(model)
result = model.transcribe(audioPath)
print('言語: ' + result["language"])
print(result["text"])

言語: ja
宮沢賢治作 雨にも負けず雨にも負けず 風にも負けず雪にも 夏の暑さにも負けぬ丈夫な体を持ち 欲はなく決して怒らず いつも静かに笑っている一日に玄米四合と味噌と少しの 野菜を食べあらゆることを自分を感情に入れ ずによく見聞きしわかりそして忘れず野原の松の林の陰の小さなかや ぶきの小屋にいて東に病気の子供あれば 行って看病してやり西に疲れた母あれば 行ってその稲の束を追い南に死にそうな人あれば 行って怖がらなくてもいいといい北に喧嘩や訴訟があれば つまらないからやめろといい緑の時は涙を流し 寒さの夏はおろおろ歩きみんなにデクノボーと呼ばれ 褒められもせず苦にもされずそういうものに私はなりたい南無変行菩薩 南常行菩薩南多方如来 南明方蓮華経南釈迦無二仏 南常行菩薩南安隆行菩薩


In [9]:
result = model.transcribe(audioPath)
print('言語: ' + result["language"])
print(result["text"])

言語: ja
宮沢賢治作 雨にも負けず雨にも負けず 風にも負けず雪にも 夏の暑さにも負けぬ丈夫な体を持ち 欲はなく決して怒らず いつも静かに笑っている一日に玄米四合と味噌と少しの 野菜を食べあらゆることを自分を感情に入れ ずによく見聞きしわかりそして忘れず野原の松の林の陰の小さなかや ぶきの小屋にいて東に病気の子供あれば 行って看病してやり西に疲れた母あれば 行ってその稲の束を追い南に死にそうな人あれば 行って怖がらなくてもいいといい北に喧嘩や訴訟があれば つまらないからやめろといい緑の時は涙を流し 寒さの夏はおろおろ歩きみんなにデクノボーと呼ばれ 褒められもせず苦にもされずそういうものに私はなりたい南無変行菩薩 南常行菩薩南多方如来 南明方蓮華経南釈迦無二仏 南常行菩薩南安隆行菩薩


## 発話の開始時間と終了時間を含めたCSVデータにする
segments のデータを csvデータにする


In [11]:
import pandas as pd
segmentsDf = pd.DataFrame.from_dict(result["segments"])

# csv ファイルに出力
segmentsDf.to_csv(outputCsvPath)

# 内容の表示
segmentsDf.head(10)

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.00,11.00,宮沢賢治作 雨にも負けず,"[50364, 39194, 3308, 95, 13352, 95, 30118, 119...",0.0,-0.212710,1.405797,0.211058
1,1,0,11.00,14.08,雨にも負けず 風にも負けず,"[50914, 35339, 4108, 4801, 11561, 254, 7625, 1...",0.0,-0.212710,1.405797,0.211058
2,2,0,14.08,19.52,雪にも 夏の暑さにも負けぬ,"[51068, 47939, 4108, 4801, 220, 42708, 2972, 2...",0.0,-0.212710,1.405797,0.211058
3,3,0,19.52,24.38,丈夫な体を持ち 欲はなく,"[51340, 39642, 3203, 29485, 5998, 17694, 6574,...",0.0,-0.212710,1.405797,0.211058
4,4,0,24.38,28.20,決して怒らず いつも静かに笑っている,"[51583, 33540, 8822, 3757, 240, 5154, 18216, 3...",0.0,-0.212710,1.405797,0.211058
5,5,2820,28.20,37.82,一日に玄米四合と味噌と少しの 野菜を食べ,"[50364, 2257, 6890, 4108, 8051, 226, 27742, 19...",0.0,-0.267053,1.261364,0.012428
6,6,2820,37.82,42.76,あらゆることを自分を感情に入れ ずに,"[50845, 3590, 5154, 34072, 4895, 13235, 5998, ...",0.0,-0.267053,1.261364,0.012428
7,7,2820,42.76,48.20,よく見聞きしわかりそして忘れず,"[51092, 5591, 6134, 10470, 25468, 7016, 2849, ...",0.0,-0.267053,1.261364,0.012428
8,8,2820,48.20,55.36,野原の松の林の陰の小さなかや ぶきの小屋にいて,"[51364, 37178, 19683, 2972, 36897, 2972, 32063...",0.0,-0.267053,1.261364,0.012428
9,9,5536,55.36,62.00,東に病気の子供あれば 行って看病してやり,"[50364, 18413, 4108, 30986, 25870, 2972, 7626,...",0.0,-0.166663,1.539773,0.117467
